# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [2]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.7 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [4]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [6]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [8]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [9]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    TABLES HERE:
    -- Table: employees
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,  -- Unique ID for each employee
    name TEXT,                -- Name of the employee
    department TEXT,          -- Department where the employee works
    hire_date DATE,           -- Date when the employee joined the company
    email TEXT,               -- Employee's email address
    phone TEXT                -- Employee's phone number
);

INSERT INTO employees VALUES
(1, 'Alice', 'HR', '2015-06-10', 'alice@example.com', '555-1234'),
(2, 'Bob', 'Engineering', '2018-09-22', 'bob@example.com', '555-5678'),
(3, 'Charlie', 'Sales', '2020-01-15', 'charlie@example.com', '555-9876'),
(4, 'Diana', 'Marketing', '2019-05-30', 'diana@example.com', '555-2468');

-- Table: salary
CREATE TABLE salary (
    ID_usr INT,              -- Employee ID from the employees table
    year INT,                -- Year of salary information
    base FLOAT,              -- Base salary of the employee
    bonus FLOAT,             -- Bonus given to the employee
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)  -- Links salary to employee
);

INSERT INTO salary VALUES
(1, 2022, 50000, 1000),
(2, 2022, 70000, 3000),
(3, 2022, 60000, 1500),
(4, 2022, 65000, 1200);

-- Table: studies
CREATE TABLE studies (
    ID INT PRIMARY KEY,      -- Unique ID for each study record
    ID_usr INT,              -- Employee ID linking to the employees table
    degree TEXT,             -- Degree earned by the employee
    institution TEXT,        -- Institution where the degree was obtained
    graduation_year INT,     -- Year the employee graduated
    major TEXT,              -- Major or field of study
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)  -- Links studies to employee
);

INSERT INTO studies VALUES
(1, 1, 'BA', 'State University', 2012, 'Human Resources'),
(2, 2, 'MSc', 'Tech Institute', 2017, 'Software Engineering'),
(3, 3, 'PhD', 'Ivy College', 2019, 'Marketing Research'),
(4, 4, 'MBA', 'Business School', 2018, 'Business Administration');

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [10]:
sp_nl2sql = sp_nl2sql.format(question="What is the average salary of employees in each department?")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    TABLES HERE:
    -- Table: employees
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,  -- Unique ID for each employee
    name TEXT,                -- Name of the employee
    department TEXT,          -- Department where the employee works
    hire_date DATE,           -- Date when the employee joined the company
    email TEXT,               -- Employee's email address
    phone TEXT                -- Employee's phone number
);

INSERT INTO employees VALUES
(1, 'Alice', 'HR', '2015-06-10', 'alice@example.com', '555-1234'),
(2, 'Bob', 'Engineering', '2018-09-22', 'bob@example.com', '555-567

In [11]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [12]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [13]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT departments.department, AVG(salary.base + salary.bonus) AS average_salary FROM employees JOIN salary ON employees.id_usr = salary.id_usr JOIN departments ON employees.department = departments.department GROUP BY departments.department ORDER BY average_salary DESC NULLS LAST;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [14]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    TABLES HERE:
    -- Table: employees
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,  -- Unique ID for each employee
    name TEXT,                -- Name of the employee
    department TEXT,          -- Department where the employee works
    hire_date DATE,           -- Date when the employee joined the company
    email TEXT,               -- Employee's email address
    phone TEXT                -- Employee's phone number
);

INSERT INTO employees VALUES
(1, 'Alice', 'HR', '2015-06-10', 'alice@example.com', '555-1234'),
(2, 'Bob', 'Engineering', '2018-09-22', 'bob@example.com', '555-5678'),
(3, 'Charlie', 'Sales', '2020-01-15', 'charlie@example.com', '555-9876'),
(4, 'Diana', 'Marketing', '2019-05-30', 'diana@example.com', '555-2468');

-- Table: salary
CREATE TABLE salary (
    ID_usr INT,              -- Employee ID from the employees table
    year INT,                -- Year of salary information
    base FLOAT,              -- Base salary of the employee
    bonus FLOAT,             -- Bonus given to the employee
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)  -- Links salary to employee
);

INSERT INTO salary VALUES
(1, 2022, 50000, 1000),
(2, 2022, 70000, 3000),
(3, 2022, 60000, 1500),
(4, 2022, 65000, 1200);

-- Table: studies
CREATE TABLE studies (
    ID INT PRIMARY KEY,      -- Unique ID for each study record
    ID_usr INT,              -- Employee ID linking to the employees table
    degree TEXT,             -- Degree earned by the employee
    institution TEXT,        -- Institution where the degree was obtained
    graduation_year INT,     -- Year the employee graduated
    major TEXT,              -- Major or field of study
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)  -- Links studies to employee
);

INSERT INTO studies VALUES
(1, 1, 'BA', 'State University', 2012, 'Human Resources'),
(2, 2, 'MSc', 'Tech Institute', 2017, 'Software Engineering'),
(3, 3, 'PhD', 'Ivy College', 2019, 'Marketing Research'),
(4, 4, 'MBA', 'Business School', 2018, 'Business Administration');

    ### Response
    Question: Return the name of the best paid employee
    ```sql
    SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    ORDER BY (s.base + s.bonus) DESC
    LIMIT 1;
    ```

    `{question}`:
    ```sql3
    """


In [15]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    TABLES HERE:
    -- Table: employees
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,  -- Unique ID for each employee
    name TEXT,                -- Name of the employee
    department TEXT,          -- Department where the employee works
    hire_date DATE,           -- Date when the employee joined the company
    email TEXT,               -- Employee's email address
    phone TEXT                -- Employee's phone number
);

INSERT INTO employees VALUES
(1, 'Alice', 'HR', '2015-06-10', 'ali

In [16]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [17]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    ORDER BY (s.base + s.bonus) DESC
    LIMIT 1;


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [18]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

     TABLES HERE:
    -- Table: employees
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,  -- Unique ID for each employee
    name TEXT,                -- Name of the employee
    department TEXT,          -- Department where the employee works
    hire_date DATE,           -- Date when the employee joined the company
    email TEXT,               -- Employee's email address
    phone TEXT                -- Employee's phone number
);

INSERT INTO employees VALUES
(1, 'Alice', 'HR', '2015-06-10', 'alice@example.com', '555-1234'),
(2, 'Bob', 'Engineering', '2018-09-22', 'bob@example.com', '555-5678'),
(3, 'Charlie', 'Sales', '2020-01-15', 'charlie@example.com', '555-9876'),
(4, 'Diana', 'Marketing', '2019-05-30', 'diana@example.com', '555-2468');

-- Table: salary
CREATE TABLE salary (
    ID_usr INT,              -- Employee ID from the employees table
    year INT,                -- Year of salary information
    base FLOAT,              -- Base salary of the employee
    bonus FLOAT,             -- Bonus given to the employee
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)  -- Links salary to employee
);

INSERT INTO salary VALUES
(1, 2022, 50000, 1000),
(2, 2022, 70000, 3000),
(3, 2022, 60000, 1500),
(4, 2022, 65000, 1200);

-- Table: studies
CREATE TABLE studies (
    ID INT PRIMARY KEY,      -- Unique ID for each study record
    ID_usr INT,              -- Employee ID linking to the employees table
    degree TEXT,             -- Degree earned by the employee
    institution TEXT,        -- Institution where the degree was obtained
    graduation_year INT,     -- Year the employee graduated
    major TEXT,              -- Major or field of study
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)  -- Links studies to employee
);

INSERT INTO studies VALUES
(1, 1, 'BA', 'State University', 2012, 'Human Resources'),
(2, 2, 'MSc', 'Tech Institute', 2017, 'Software Engineering'),
(3, 3, 'PhD', 'Ivy College', 2019, 'Marketing Research'),
(4, 4, 'MBA', 'Business School', 2018, 'Business Administration');

    ### Samples

    Question: Return the name of the best paid employee
    ```sql
    SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    ORDER BY (s.base + s.bonus) DESC
    LIMIT 1;
    ```

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [19]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

     TABLES HERE:
    -- Table: employees
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,  -- Unique ID for each employee
    name TEXT,                -- Name of the employee
    department TEXT,          -- Department where the employee works
    hire_date DATE,           -- Date when the employee joined the company
    email TEXT,               -- Employee's email address
    phone TEXT                -- Employee's phone number
);

INSERT INTO employees VALUES
(1, 'Alice', 'HR', '2015-06-10', 'ali

In [20]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [21]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    ORDER BY (s.base + s.bonus) DESC
    LIMIT 1;


#Now the question in arabic.


In [22]:
sp_nl2sql3 = sp_nl2sql3b.format(question="من هو أعلى الموظفين أجرًا؟")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

     TABLES HERE:
    -- Table: employees
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,  -- Unique ID for each employee
    name TEXT,                -- Name of the employee
    department TEXT,          -- Department where the employee works
    hire_date DATE,           -- Date when the employee joined the company
    email TEXT,               -- Employee's email address
    phone TEXT                -- Employee's phone number
);

INSERT INTO employees VALUES
(1, 'Alice', 'HR', '2015-06-10', 'ali

In [23]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [24]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    ORDER BY (s.base + s.bonus) DESC
    LIMIT 1;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

##**Version1: Prompt without Shots.**

In [25]:
sp_nl2sv1 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

     TABLES HERE:
    -- Table: employees
CREATE TABLE employees (
    emp_id INT PRIMARY KEY,    -- Unique ID for each employee
    emp_name TEXT,             -- Name of the employee
    team TEXT,                 -- Team or department
    join_date DATE             -- Date the employee joined
);

INSERT INTO employees VALUES
(1, 'Olivia Grant', 'Design', '2021-04-10'),
(2, 'James Holt', 'Development', '2020-09-15'),
(3, 'Ava Ruiz', 'Marketing', '2022-01-20'),
(4, 'Ethan Cole', 'Support', '2021-11-05');

-- Table: tasks
CREATE TABLE tasks (
    task_id INT PRIMARY KEY,   -- Unique ID for each task
    emp_id INT,                -- Employee assigned to the task
    task_description TEXT,     -- Description of the task
    status TEXT,               -- Status: 'Pending', 'In Progress', 'Completed'
    due_date DATE,             -- Task due date
    FOREIGN KEY (emp_id) REFERENCES employees(emp_id)  -- Links tasks to employees
);

INSERT INTO tasks VALUES
(1, 1, 'Create logo mockups', 'Completed', '2023-05-01'),
(2, 1, 'Review client feedback', 'In Progress', '2023-06-15'),
(3, 2, 'Code new feature', 'Completed', '2023-04-20'),
(4, 2, 'Fix bugs in app', 'Completed', '2023-05-10'),
(5, 3, 'Draft campaign plan', 'Pending', '2023-07-01'),
(6, 4, 'Answer support tickets', 'Completed', '2023-05-15');

-- Table: progress_logs
CREATE TABLE progress_logs (
    log_id INT PRIMARY KEY,    -- Unique ID for each log entry
    task_id INT,               -- Task being logged
    log_date DATE,             -- Date of the progress update
    note TEXT,                 -- Progress note or comment
    FOREIGN KEY (task_id) REFERENCES tasks(task_id)  -- Links logs to tasks
);

INSERT INTO progress_logs VALUES
(1, 1, '2023-04-25', 'Initial drafts done'),
(2, 3, '2023-04-18', 'Feature tested successfully'),
(3, 4, '2023-05-08', 'Bugs resolved'),
(4, 6, '2023-05-14', 'Tickets cleared');

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

sp_nl2sv1 = sp_nl2sv1.format(question="Return the name of the employee with the most recent progress update ")

input_sentences = tokenizer(sp_nl2sv1, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")


WITH task_progress AS (SELECT t.task_id, p.log_date, row_number() OVER(PARTITION BY t.task_id ORDER BY p.log_date DESC) AS row_num FROM tasks t JOIN progress_logs p ON t.task_id = p.task_id) SELECT e.emp_name FROM employees e JOIN task_progress tp ON e.emp_id = tp.task_id WHERE tp.row_num = 1;


##**Version2: Prompt with shots OpenAI Style.**

In [26]:
sp_nl2sv2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

     TABLES HERE:
    -- Table: employees
CREATE TABLE employees (
    emp_id INT PRIMARY KEY,    -- Unique ID for each employee
    emp_name TEXT,             -- Name of the employee
    team TEXT,                 -- Team or department
    join_date DATE             -- Date the employee joined
);

INSERT INTO employees VALUES
(1, 'Olivia Grant', 'Design', '2021-04-10'),
(2, 'James Holt', 'Development', '2020-09-15'),
(3, 'Ava Ruiz', 'Marketing', '2022-01-20'),
(4, 'Ethan Cole', 'Support', '2021-11-05');

-- Table: tasks
CREATE TABLE tasks (
    task_id INT PRIMARY KEY,   -- Unique ID for each task
    emp_id INT,                -- Employee assigned to the task
    task_description TEXT,     -- Description of the task
    status TEXT,               -- Status: 'Pending', 'In Progress', 'Completed'
    due_date DATE,             -- Task due date
    FOREIGN KEY (emp_id) REFERENCES employees(emp_id)  -- Links tasks to employees
);

INSERT INTO tasks VALUES
(1, 1, 'Create logo mockups', 'Completed', '2023-05-01'),
(2, 1, 'Review client feedback', 'In Progress', '2023-06-15'),
(3, 2, 'Code new feature', 'Completed', '2023-04-20'),
(4, 2, 'Fix bugs in app', 'Completed', '2023-05-10'),
(5, 3, 'Draft campaign plan', 'Pending', '2023-07-01'),
(6, 4, 'Answer support tickets', 'Completed', '2023-05-15');

-- Table: progress_logs
CREATE TABLE progress_logs (
    log_id INT PRIMARY KEY,    -- Unique ID for each log entry
    task_id INT,               -- Task being logged
    log_date DATE,             -- Date of the progress update
    note TEXT,                 -- Progress note or comment
    FOREIGN KEY (task_id) REFERENCES tasks(task_id)  -- Links logs to tasks
);

INSERT INTO progress_logs VALUES
(1, 1, '2023-04-25', 'Initial drafts done'),
(2, 3, '2023-04-18', 'Feature tested successfully'),
(3, 4, '2023-05-08', 'Bugs resolved'),
(4, 6, '2023-05-14', 'Tickets cleared');

    ### Response

    Question: Return the name of the employee with the most recent progress update
    ```sql
    SELECT e.emp_name
    FROM employees e
    JOIN tasks t ON e.emp_id = t.emp_id
    JOIN progress_logs pl ON t.task_id = pl.task_id
    ORDER BY pl.log_date DESC
    LIMIT 1;
    ```

    `{question}`:
    ```sql3
    """

sp_nl2sv2 = sp_nl2sv2.format(question="Return the name of the employee with the most recent progress update ")

input_sentences = tokenizer(sp_nl2sv2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.emp_name
    FROM employees e
    JOIN tasks t ON e.emp_id = t.emp_id
    JOIN progress_logs pl ON t.task_id = pl.task_id
    ORDER BY pl.log_date DESC
    LIMIT 1;


##**Version3: Prompt with Shots in Sample Style.**

In [29]:
sp_nl2sv3 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

     TABLES HERE:
    -- Table: employees
CREATE TABLE employees (
    emp_id INT PRIMARY KEY,    -- Unique ID for each employee
    emp_name TEXT,             -- Name of the employee
    team TEXT,                 -- Team or department
    join_date DATE             -- Date the employee joined
);

INSERT INTO employees VALUES
(1, 'Olivia Grant', 'Design', '2021-04-10'),
(2, 'James Holt', 'Development', '2020-09-15'),
(3, 'Ava Ruiz', 'Marketing', '2022-01-20'),
(4, 'Ethan Cole', 'Support', '2021-11-05');

-- Table: tasks
CREATE TABLE tasks (
    task_id INT PRIMARY KEY,   -- Unique ID for each task
    emp_id INT,                -- Employee assigned to the task
    task_description TEXT,     -- Description of the task
    status TEXT,               -- Status: 'Pending', 'In Progress', 'Completed'
    due_date DATE,             -- Task due date
    FOREIGN KEY (emp_id) REFERENCES employees(emp_id)  -- Links tasks to employees
);

INSERT INTO tasks VALUES
(1, 1, 'Create logo mockups', 'Completed', '2023-05-01'),
(2, 1, 'Review client feedback', 'In Progress', '2023-06-15'),
(3, 2, 'Code new feature', 'Completed', '2023-04-20'),
(4, 2, 'Fix bugs in app', 'Completed', '2023-05-10'),
(5, 3, 'Draft campaign plan', 'Pending', '2023-07-01'),
(6, 4, 'Answer support tickets', 'Completed', '2023-05-15');

-- Table: progress_logs
CREATE TABLE progress_logs (
    log_id INT PRIMARY KEY,    -- Unique ID for each log entry
    task_id INT,               -- Task being logged
    log_date DATE,             -- Date of the progress update
    note TEXT,                 -- Progress note or comment
    FOREIGN KEY (task_id) REFERENCES tasks(task_id)  -- Links logs to tasks
);

INSERT INTO progress_logs VALUES
(1, 1, '2023-04-25', 'Initial drafts done'),
(2, 3, '2023-04-18', 'Feature tested successfully'),
(3, 4, '2023-05-08', 'Bugs resolved'),
(4, 6, '2023-05-14', 'Tickets cleared');

    ### Sample

    Question: Return the name of the employee with the most recent progress update
    ```sql
    SELECT e.emp_name
    FROM employees e
    JOIN tasks t ON e.emp_id = t.emp_id
    JOIN progress_logs pl ON t.task_id = pl.task_id
    ORDER BY pl.log_date DESC
    LIMIT 1;
    ```

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

sp_nl2sv3 = sp_nl2sv3.format(question="Return the name of the employee with the most recent progress update")

input_sentences = tokenizer(sp_nl2sv3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.emp_name
    FROM employees
    JOIN tasks ON employees.emp_id = tasks.emp_id
    JOIN progress_logs ON tasks.task_id = progress_logs.task_id
    ORDER BY progress_logs.log_date DESC
    LIMIT 1;


#**Try The Question in Other Language.**

In [30]:
sp_nl2sv4 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

     TABLES HERE:
    -- Table: employees
CREATE TABLE employees (
    emp_id INT PRIMARY KEY,    -- Unique ID for each employee
    emp_name TEXT,             -- Name of the employee
    team TEXT,                 -- Team or department
    join_date DATE             -- Date the employee joined
);

INSERT INTO employees VALUES
(1, 'Olivia Grant', 'Design', '2021-04-10'),
(2, 'James Holt', 'Development', '2020-09-15'),
(3, 'Ava Ruiz', 'Marketing', '2022-01-20'),
(4, 'Ethan Cole', 'Support', '2021-11-05');

-- Table: tasks
CREATE TABLE tasks (
    task_id INT PRIMARY KEY,   -- Unique ID for each task
    emp_id INT,                -- Employee assigned to the task
    task_description TEXT,     -- Description of the task
    status TEXT,               -- Status: 'Pending', 'In Progress', 'Completed'
    due_date DATE,             -- Task due date
    FOREIGN KEY (emp_id) REFERENCES employees(emp_id)  -- Links tasks to employees
);

INSERT INTO tasks VALUES
(1, 1, 'Create logo mockups', 'Completed', '2023-05-01'),
(2, 1, 'Review client feedback', 'In Progress', '2023-06-15'),
(3, 2, 'Code new feature', 'Completed', '2023-04-20'),
(4, 2, 'Fix bugs in app', 'Completed', '2023-05-10'),
(5, 3, 'Draft campaign plan', 'Pending', '2023-07-01'),
(6, 4, 'Answer support tickets', 'Completed', '2023-05-15');

-- Table: progress_logs
CREATE TABLE progress_logs (
    log_id INT PRIMARY KEY,    -- Unique ID for each log entry
    task_id INT,               -- Task being logged
    log_date DATE,             -- Date of the progress update
    note TEXT,                 -- Progress note or comment
    FOREIGN KEY (task_id) REFERENCES tasks(task_id)  -- Links logs to tasks
);

INSERT INTO progress_logs VALUES
(1, 1, '2023-04-25', 'Initial drafts done'),
(2, 3, '2023-04-18', 'Feature tested successfully'),
(3, 4, '2023-05-08', 'Bugs resolved'),
(4, 6, '2023-05-14', 'Tickets cleared');

    ### Sample

    Question: Return the name of the employee with the most recent progress update
    ```sql
    SELECT e.emp_name
    FROM employees e
    JOIN tasks t ON e.emp_id = t.emp_id
    JOIN progress_logs pl ON t.task_id = pl.task_id
    ORDER BY pl.log_date DESC
    LIMIT 1;
    ```

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

sp_nl2sv4 = sp_nl2sv4.format(question="返回最近一次进度更新的员工姓名")

input_sentences = tokenizer(sp_nl2sv4, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.emp_name
     FROM employees
     JOIN tasks ON employees.emp_id = tasks.emp_id
     JOIN progress_logs ON tasks.task_id = progress_logs.task_id
     ORDER BY progress_logs.log_date DESC
     LIMIT 1;


#**SQL Query Generation Using defog/sqlcoder-7b Across Prompt Versions**

#### **Purpose**
This experiment evaluated "defog/sqlcoder-7b," a Hugging Face model based on Mistral and fine-tuned for SQL generation, in converting natural language questions into SQL queries across four prompt versions. The question tested was "Return the name of the employee with the most recent progress update," except in Version 4, where it was in Chinese ("返回最近一次进度更新的员工姓名"). The database schema included three tables—`employees`, `tasks`, and `progress_logs`—linked by foreign keys. The goal was to assess how prompt structure, example inclusion (shots), and language variation affect the model’s SQL output accuracy.

#### **Findings Across Versions**

##### **Version 1: Prompt Without Shots**
- **Performance**: In this zero-shot scenario, the model overcomplicated the query with a CTE and `ROW_NUMBER()`, but it contains an error: the join `e.emp_id = tp.task_id` is incorrect (it should be `t.emp_id`). This hallucination shows that, despite fine-tuning, sqlcoder-7b struggles without guidance, producing an invalid result.

##### **Version 2: Prompt with Shots (OpenAI Style)**
- **Performance**: With an inline example, the model generated a correct, concise query, properly joining tables, sorting by `log_date`, and limiting to one result. The shot effectively guided its fine-tuned SQL skills, avoiding errors.

##### **Version 3: Prompt with Shots (Sample Style)**
- **Performance**: Using a separated "Sample" section, the model produced a correct query, slightly varying in style (no aliases) from Version 2 but functionally identical. The example ensured accuracy, leveraging its SQL expertise.

##### **Version 4: Prompt with Shots in Another Language (Chinese)**
- **Performance**: With a Chinese question and an English example, the model generated a correct query, matching Version 3. This suggests sqlcoder-7b can process multilingual inputs effectively when supported by a clear example.

#### **Variations That Didn’t Work Well**
- **Version 1**: The output was incorrect due to a join error (`e.emp_id = tp.task_id` instead of `t.emp_id`), producing an invalid query. This highlights a significant weakness in zero-shot scenarios, where the model overcomplicates and hallucinates despite its SQL fine-tuning.
- Versions 2, 3, and 4 were all correct, with no notable failures. The presence of shots consistently prevented errors, aligning the model’s output with the schema and question.

#### **Differences Between Version 2 and Version 3**
- **Structure**: Version 2 places the shot inline in the "Response" section, while Version 3 isolates it in a "Sample" section.
- **Impact**: Version 2 uses aliases (e.g., `e.emp_name`), while Version 3 uses full table names (e.g., `employees.emp_name`), but both are correct and equivalent. Version 2’s inline style offers immediate context, while Version 3’s separation might enhance readability or scalability.
- **Purpose**: Version 2 targets quick, conversational generation; Version 3 provides a structured, reusable reference, though both succeed here.

#### **What I Learned**
- **Shots Prevent Errors**: Version 1’s failure versus the success of Versions 2, 3, and 4 shows that examples are crucial to avoid hallucination, even in a fine-tuned SQL model.
- **Fine-Tuning Isn’t Enough**: Sqlcoder-7b’s zero-shot error (Version 1) reveals that its SQL expertise requires guidance to avoid overcomplication and mistakes.
- **Prompt Style Flexibility**: The model adapted to inline (Version 2) and separated (Version 3) shots, producing correct queries with minor stylistic differences, offering design versatility.
- **Multilingual Capability**: Version 4’s success with Chinese input suggests sqlcoder-7b handles non-English questions well with examples, though its Mistral base may limit broader language proficiency.

#### **Conclusion**
Sqlcoder-7b performed flawlessly in Versions 2, 3, and 4 with shots, generating accurate SQL queries, but failed in Version 1’s zero-shot attempt due to a join error. Versions 2 and 3, differing in shot placement and style, both succeeded, offering conversational versus structured options. Version 4’s correct output in Chinese highlights multilingual potential when guided. The model’s fine-tuning excels with examples, but zero-shot scenarios expose weaknesses, emphasizing the need for shots in reliable SQL generation.